In [4]:
# Импортируем библотеки
import pandas as pd
import numpy as np

In [20]:
# Считываем данные
data = pd.read_csv('train.csv')

In [21]:
# Посмотрим на данные
data

,index,name,description,target
0,324865089,продавец кассир,<strong>Обязанности:</strong> <ul> <li>работа ...,-1
1,169467135,продавец мила (шевченко 17),<p><strong>Магазин МИЛА по адресу б-р Шевченко...,-1
2,169939030,кассир в пиццерию г витебск,"<p><strong>Устал искать работу? Может, хочешь ...",5223
3,31956044,начальник строительного участка,<p>Компании &quot;Артель-Строй&quot; на постоя...,-1
4,36781653,продавец кассир (санкт петербург бухарестская 31),<p>Обязанности:</p><p> •Обслуживание покупа...,-1
...,...,...,...,...
29995,50002587,офис менеджер,<p><strong>Обязанности:</strong></p> <ul> <li>...,3341
29996,126009334,дворник (разнорабочий),"<p>В группу компаний ООО ""Ю-МЕТ"" требуется раз...",-1
29997,183546781,офис менеджер,<strong>Обязанности:</strong> <ul> <li>прием и...,3341
29998,123274416,разнорабочий,"Производить распиловку бревен, брусьев и загот...",-1


In [22]:
# Посмотрим количество текстов пренадлежащих разным классам
data['target'].value_counts()

-1       14350
 6121     1593
 5223     1328
 9613      641
 2411      578
         ...  
 1345        1
 3342        1
 5222        1
 7411        1
 3334        1
Name: target, Length: 131, dtype: int64

## Построим первичную модель

In [5]:
# Импортируем нужные билиотеки
import pandas as pd
import nltk
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from pymystem3 import Mystem

In [23]:
def preprocess_text(text): 
    # удаление HTML/XML тэгов
    text = re.sub('<[^>]+>', '', text)

    # удаление символов кроме букв
    text = re.sub('[^a-zA-Zа-яА-Я]', ' ', text)
    
    # приведение текста к нижнему регистру
    text = text.lower()
    
    # токенизация
    words = word_tokenize(text)

    # Удаление стоп слов
    stop_words = set(stopwords.words('russian'))
    words = [w for w in words if w not in stop_words]

    # лемматизация
    m = Mystem()
    lemmatized_words = [''.join(m.lemmatize(w)).strip() for w in words]

    return ' '.join(lemmatized_words)

In [ ]:
%%time
# Загружаем данные
data = pd.read_csv('train.csv', index_col='index')

# комбинируем название и описание в один текстовый столбец
data['text'] = data['name'].fillna('') + ' ' + data['description'].fillna('')
data.drop(['name', 'description'], axis=1, inplace=True)

# Обработка текста
data['text'] = data['text'].apply(preprocess_text)

In [9]:
data.head()

,target,text
index,,
324865089,-1,продавец кассир обязанности работа кассой откр...
169467135,-1,продавец мила шевченко магазин мила адресу б р...
169939030,5223,кассир пиццерию г витебск устал искать работу ...
31956044,-1,начальник строительного участка компании quot ...
36781653,-1,продавец кассир санкт петербург бухарестская о...


In [10]:
# Делим данные на тренировочный и тестовый датасет
X_train, X_val, y_train, y_val = train_test_split(data['text'], data['target'], test_size=0.2, random_state=42)

In [ ]:
# Делим данные на тренировочный и тестовый датасет
X_train, X_val, y_train, y_val = train_test_split(data['text'], data['target'], test_size=0.2, random_state=42)

# Создаем цепочку из векторизатора и модели логистической регрессии
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', LogisticRegression(solver='liblinear')),
])

# Тренируем модель
pipeline.fit(X_train, y_train)

# Оцениваем модель на валидационом датасете
y_pred = pipeline.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print(classification_report(y_val, y_pred))

## Используем предобученные векторизаторы

In [10]:
# Устанавливаем библиотеку
if 0:
    !pip install fasttext

In [1]:
# Импортируем библиотеки
import fasttext
import fasttext.util

# Скачиваем русскую модель
fasttext.util.download_model('ru', if_exists='ignore')

'cc.ru.300.bin'

In [2]:
# Загружаем готовые FastText числовые векторы
fasttext_model = fasttext.load_model('cc.ru.300.bin')

def fasttext_vectorizer(texts):
    '''
    Данная функция векторизует текстовые данные с помощью предобученной модели FastText, 
    которая преобразует тексты в векторы фиксированной размерности. 
    Входным параметром является список текстов, на выходе функция возвращает массив с векторными представлениями текстов, 
    где каждая строка соответствует одному тексту. Каждый вектор имеет размерность, 
    равную размерности пространства эмбеддингов, которое определяется моделью FastText.'''
    embeddings = np.zeros((len(texts), fasttext_model.get_dimension()))
    for i, text in enumerate(texts):
        embeddings[i] = fasttext_model.get_sentence_vector(text)
    return embeddings

In [7]:
vect_text = fasttext_vectorizer('работа рядом домом учебой график стабильная выплата заработной платы раза месяц')
vect_text

array([[-0.03168774, -0.05454991,  0.01556752, ...,  0.11077441,
         0.03907987, -0.05253111],
       [-0.00180265, -0.04640927, -0.01071023, ...,  0.05638637,
        -0.09506392, -0.13391244],
       [ 0.03945547, -0.03289286,  0.01155682, ...,  0.01053472,
        -0.04632451, -0.0313887 ],
       ...,
       [ 0.0173933 ,  0.01945579,  0.02562522, ..., -0.06275136,
         0.0536981 , -0.05274193],
       [ 0.02949647, -0.02347934, -0.00222554, ...,  0.02867848,
        -0.12651373, -0.03364033],
       [-0.00224334, -0.03744297, -0.10470287, ...,  0.03128449,
         0.0137482 , -0.1214609 ]])

In [13]:
# Применение функций векторизаторов к данным
X_train_embeddings = fasttext_vectorizer(X_train)
X_val_embeddings = fasttext_vectorizer(X_val)

# Тренируем модель Логистической Регрессии
classifier = LogisticRegression(solver='liblinear')
classifier.fit(X_train_embeddings, y_train)

# Оцениваем модель на валидационом датасете
y_pred = classifier.predict(X_val_embeddings)
accuracy = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print(classification_report(y_val, y_pred))


Accuracy: 0.48
              precision    recall  f1-score   support

          -1       0.48      0.96      0.64      2870
        1120       0.00      0.00      0.00        86
        1211       0.00      0.00      0.00         2
        1213       0.00      0.00      0.00        35
        1219       0.00      0.00      0.00         8
        1221       0.00      0.00      0.00        43
        1321       0.00      0.00      0.00         4
        1323       0.00      0.00      0.00         9
        1324       0.00      0.00      0.00         1
        1325       0.00      0.00      0.00         2
        1342       0.00      0.00      0.00         1
        1349       0.00      0.00      0.00        35
        1420       0.00      0.00      0.00        40
        2149       0.00      0.00      0.00        10
        2211       1.00      0.02      0.03        59
        2240       0.00      0.00      0.00        35
        2269       0.00      0.00      0.00        28
        2411

C:\Users\Nikit\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Nikit\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Nikit\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
# Создаем копию датасета 
data_new = data.copy()

In [13]:
# Создадим датасет, где у нас у класса минимум 3 элемента
col = data_new['target'].value_counts()
bad_classes = col.loc[col < 3].index

In [14]:
# Присвоим -2
data_new.loc[data_new['target'].isin(bad_classes), 'target'] = -2

In [15]:
# Смотрим на результат
data_new['target'].value_counts()

-1       14350
 6121     1593
 5223     1328
 9613      641
 2411      578
         ...  
 8111        3
 1223        3
 3111        3
 2413        3
 7521        3
Name: target, Length: 93, dtype: int64

In [16]:
# Уберем класс -1 и -2
data_new = data_new.loc[(data_new['target'] != -1) & (data_new['target'] != -2)]

In [ ]:
from catboost import CatBoostClassifier
from sklearn.utils import resample
import pickle

# Делим данные на тренировочную и тестовую выборку
X_train, X_val, y_train, y_val = train_test_split(data_new['text'], data_new['target'], test_size=0.2, 
                                                  random_state=19, stratify=data_new['target'])

# Конвертируем текста в эмбединги
X_train_embeddings = fasttext_vectorizer(X_train)
X_val_embeddings = fasttext_vectorizer(X_val)

# Тренируем CatBoost classifier
classifier = CatBoostClassifier(iterations=100, eval_metric='Accuracy', use_best_model=True, random_seed=19)
classifier.fit(X_train_embeddings, y_train, eval_set=(X_val_embeddings, y_val))

# Сохраняем обученную модель с использованием pickle
pickle_file = "catboost_classifier.pkl"
with open(pickle_file, "wb") as f:
    pickle.dump(classifier, f)

# Оценим модель
y_pred = classifier.predict(X_val_embeddings)
accuracy = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print(classification_report(y_val, y_pred))